In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
import pandas as pd

2021-08-24 12:21:28,893 : INFO : NumExpr defaulting to 4 threads.


In [3]:
import simplejson


def json_save(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        simplejson.dump(data, f, separators=(',', ':'), iterable_as_array=True)

In [ ]:
# df_train = pd.read_csv('./train.csv')
# df_test = pd.read_csv('./test.csv')

In [4]:
df_full = pd.read_csv('./full.csv')

In [ ]:
# # dev
# df_train = df_train.sample(50, random_state=42)
# df_test = df_test.sample(50, random_state=42)

In [ ]:
# df_train.shape, df_test.shape

In [5]:
df_full.shape

(490, 7)

In [6]:
from tqdm.notebook import tqdm_notebook

In [7]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

def corpus2tokens(raw_corpus):
    return [
        [
            wd
            for wd in simple_preprocess(doc)
            if not wd in STOPWORDS
        ]
        for doc in tqdm_notebook(raw_corpus)
    ]

In [ ]:
# %%time
# corpus_words_train = corpus2tokens(df_train.text.to_list())

In [8]:
%%time
corpus_words_full = corpus2tokens(df_full.text.to_list())


CPU times: user 1.08 s, sys: 74 ms, total: 1.15 s
Wall time: 1.76 s


In [ ]:
# json_save(corpus_words_train, './lda.corpus.train.json')

In [9]:
json_save(corpus_words_full, './lda.corpus.full.json')

In [ ]:
# %%time
# from gensim import corpora

# dictionary = corpora.Dictionary(corpus_words_train)
# dictionary.filter_extremes()
# dictionary.save_as_text('./lda.dictionary.txt')

In [10]:
%%time
from gensim import corpora

dictionary = corpora.Dictionary(corpus_words_full)
dictionary.filter_extremes()
dictionary.save_as_text('./lda.dictionary.txt')

2021-08-24 12:23:06,823 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-08-24 12:23:07,353 : INFO : built Dictionary(14438 unique tokens: ['added', 'administrator', 'affairs', 'ag', 'air']...) from 490 documents (total 148823 corpus positions)
2021-08-24 12:23:07,428 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(14438 unique tokens: ['added', 'administrator', 'affairs', 'ag', 'air']...) from 490 documents (total 148823 corpus positions)", 'datetime': '2021-08-24T12:23:07.358750', 'gensim': '4.0.1', 'python': '3.8.10 (default, Jun  4 2021, 15:09:15) \n[GCC 7.5.0]', 'platform': 'Linux-5.13.8-100.fc33.x86_64-x86_64-with-glibc2.17', 'event': 'created'}
2021-08-24 12:23:07,580 : INFO : discarding 10533 tokens: [('air', 263), ('airlines', 263), ('amend', 2), ('anger', 4), ('annie', 1), ('applicable', 2), ('baby', 2), ('beaune', 4), ('beta', 4), ('bitter', 2)]...
2021-08-24 12:23:07,581 : INFO : keeping 3905 tokens which were in no less than 5 and no more t

CPU times: user 622 ms, sys: 60.3 ms, total: 683 ms
Wall time: 876 ms


In [ ]:
# %%time
# corpus_train = [dictionary.doc2bow(text) for text in corpus_words_train]
# corpora.MmCorpus.serialize('./lda.corpus.txt', corpus_train)

In [12]:
%%time
corpus_full = [dictionary.doc2bow(text) for text in corpus_words_full]
corpora.MmCorpus.serialize('./lda.corpus.txt', corpus_full)

2021-08-24 12:23:47,224 : INFO : storing corpus in Matrix Market format to ./lda.corpus.txt
2021-08-24 12:23:47,226 : INFO : saving sparse matrix to ./lda.corpus.txt
2021-08-24 12:23:47,228 : INFO : PROGRESS: saving document #0
2021-08-24 12:23:47,487 : INFO : saved 490x3905 matrix, density=4.225% (80835/1913450)
2021-08-24 12:23:47,489 : INFO : saving MmCorpus index to ./lda.corpus.txt.index


CPU times: user 417 ms, sys: 58.9 ms, total: 476 ms
Wall time: 548 ms


In [13]:
from gensim import models

In [ ]:
# %%time
# from gensim import models

# lda = models.ldamulticore.LdaMulticore(
#     corpus_train,
#     id2word=dictionary, 
#     num_topics=20,
#     chunksize=4000,
#     passes=20,
#     workers=64
# )
# lda.save("./lda.model")

In [14]:
%%time
from gensim import models

lda = models.ldamulticore.LdaMulticore(
    corpus_full,
    id2word=dictionary, 
    num_topics=20,
    chunksize=4000,
    passes=20,
    workers=3  #64
)
lda.save("./lda.model")

2021-08-24 12:24:18,450 : INFO : using symmetric alpha at 0.05
2021-08-24 12:24:18,451 : INFO : using symmetric eta at 0.05
2021-08-24 12:24:18,453 : INFO : using serial LDA version on this node
2021-08-24 12:24:18,462 : INFO : running online LDA training, 20 topics, 20 passes over the supplied corpus of 490 documents, updating every 12000 documents, evaluating every ~490 documents, iterating 50x with a convergence threshold of 0.001000
2021-08-24 12:24:18,466 : INFO : training LDA model using 3 processes
2021-08-24 12:24:18,503 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #490/490, outstanding queue size 1
2021-08-24 12:24:19,584 : INFO : topic #4 (0.050): 0.005*"airline" + 0.005*"billion" + 0.005*"people" + 0.004*"vaccine" + 0.004*"company" + 0.004*"demand" + 0.004*"million" + 0.004*"virus" + 0.004*"executive" + 0.004*"country"
2021-08-24 12:24:19,586 : INFO : topic #6 (0.050): 0.008*"people" + 0.006*"cases" + 0.006*"government" + 0.005*"country" + 0.005*"vaccine"

2021-08-24 12:24:26,916 : INFO : topic #19 (0.050): 0.013*"singapore" + 0.009*"belarus" + 0.008*"delta" + 0.008*"people" + 0.008*"quarantine" + 0.008*"alexander" + 0.008*"testing" + 0.007*"monday" + 0.007*"international" + 0.006*"flight"
2021-08-24 12:24:26,917 : INFO : topic #15 (0.050): 0.009*"city" + 0.007*"billion" + 0.007*"airport" + 0.006*"mexico" + 0.005*"million" + 0.005*"people" + 0.005*"india" + 0.005*"infections" + 0.004*"debt" + 0.004*"wave"
2021-08-24 12:24:26,919 : INFO : topic #7 (0.050): 0.019*"million" + 0.014*"airline" + 0.011*"lufthansa" + 0.009*"norwegian" + 0.009*"billion" + 0.008*"company" + 0.008*"euros" + 0.008*"people" + 0.007*"aid" + 0.006*"carrier"
2021-08-24 12:24:26,920 : INFO : topic #4 (0.050): 0.018*"billion" + 0.011*"airline" + 0.009*"qantas" + 0.009*"company" + 0.008*"business" + 0.007*"cathay" + 0.006*"loss" + 0.006*"carrier" + 0.006*"shares" + 0.006*"international"
2021-08-24 12:24:26,922 : INFO : topic diff=1.100098, rho=0.404144
2021-08-24 12:24:27

2021-08-24 12:24:34,828 : INFO : topic diff=0.352099, rho=0.299846
2021-08-24 12:24:35,507 : INFO : -7.717 per-word bound, 210.4 perplexity estimate based on a held-out corpus of 490 documents with 118829 words
2021-08-24 12:24:35,508 : INFO : PROGRESS: pass 11, dispatched chunk #0 = documents up to #490/490, outstanding queue size 1
2021-08-24 12:24:36,128 : INFO : topic #3 (0.050): 0.010*"international" + 0.009*"countries" + 0.008*"carriers" + 0.007*"europe" + 0.007*"capacity" + 0.007*"airport" + 0.006*"group" + 0.006*"summer" + 0.006*"levels" + 0.006*"week"
2021-08-24 12:24:36,129 : INFO : topic #7 (0.050): 0.021*"million" + 0.017*"lufthansa" + 0.017*"airline" + 0.015*"billion" + 0.011*"euros" + 0.010*"aid" + 0.009*"carrier" + 0.009*"company" + 0.009*"norwegian" + 0.009*"government"
2021-08-24 12:24:36,131 : INFO : topic #14 (0.050): 0.011*"eu" + 0.010*"emissions" + 0.007*"carbon" + 0.007*"industry" + 0.006*"fuel" + 0.006*"market" + 0.005*"flights" + 0.005*"aviation" + 0.005*"europe

2021-08-24 12:24:42,663 : INFO : topic #18 (0.050): 0.016*"space" + 0.012*"flight" + 0.010*"virgin" + 0.010*"biden" + 0.008*"president" + 0.008*"galactic" + 0.007*"branson" + 0.006*"passengers" + 0.005*"trump" + 0.005*"house"
2021-08-24 12:24:42,664 : INFO : topic #7 (0.050): 0.020*"million" + 0.019*"billion" + 0.019*"lufthansa" + 0.018*"airline" + 0.012*"euros" + 0.011*"carrier" + 0.011*"government" + 0.010*"company" + 0.010*"aid" + 0.009*"norwegian"
2021-08-24 12:24:42,665 : INFO : topic diff=0.129337, rho=0.241666
2021-08-24 12:24:43,291 : INFO : -7.671 per-word bound, 203.8 perplexity estimate based on a held-out corpus of 490 documents with 118829 words
2021-08-24 12:24:43,292 : INFO : PROGRESS: pass 17, dispatched chunk #0 = documents up to #490/490, outstanding queue size 1
2021-08-24 12:24:43,976 : INFO : topic #16 (0.050): 0.033*"boeing" + 0.030*"max" + 0.025*"aircraft" + 0.016*"jets" + 0.014*"planes" + 0.012*"united" + 0.011*"plane" + 0.009*"airline" + 0.008*"order" + 0.008*"

CPU times: user 43.6 s, sys: 2.14 s, total: 45.7 s
Wall time: 28.9 s


In [15]:
# %%time
# text_test = corpus2tokens(df_test.text.to_list())

In [16]:
# json_save(text_test, './lda.corpus.test.json')

In [17]:
# df_production = pd.read_csv('../data/BBairline200722.csv')

In [18]:
# df_production.head()

In [19]:
# %%time
# text_production = corpus2tokens(df_production["5"].to_list())

In [20]:
# json_save(text_production, './lda.corpus.production.json')